In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler

In [2]:
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,  # 창의적인 정도, 0~1 : 1에 가까울수록 창의적이고 무작위성을 가짐
    # max_tokens=5000,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [17]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class internataional chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [18]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)
# You don't radically modify the recipe. -> chef_prompt에서 전달받은 초기 레시피를 완전히 바꾸는 경우를 방지
# If there is no alternative for a food just say you don't know to replace it. -> hallucination 방지 문장

veg_chain = veg_chef_prompt | chat

In [20]:
# final_chain = chef_chain | veg_chain

# final_chain.invoke({
#     "cuisine" : "indian",
# })

# 입력값은 cuisine, recipe
# cuisine은 chef_chain에 입력되고 그 output이 veg_chain의 recipe로 입력되어야 함
# 그러기 위해서는 chef_chain의 output이 recipe라는 걸 알려줘야 함

final_chain = {"recipe": chef_chain} | veg_chain
# 이렇게 해서 chain 간에 context object를 입력한 것
# {"recipe": chef_chain} 형태는 lagnchain의 언어 같은거 : RunnableMap -> 응답 결과를 다음 체인으로 전달
# invoke는 해당 체인에 인자 값을 넣어주는 역할

final_chain.invoke(
    {
        "cuisine": "indian",
    }
)

Great choice! How about trying a classic Indian dish like Chicken Tikka Masala? Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb (450g) boneless, skinless chicken thighs, cut into bite-sized pieces
- 1 cup plain yogurt
- 3 cloves garlic, minced
- 1 tbsp grated fresh ginger
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp turmeric
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a large bowl, combine the yogurt, garlic, ginger, garam masala, cumin, coriander, turmeric, cayenne pepper, and salt. Add the chicken pieces and toss to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.

2. Preheat the broiler to high. Thread the marinated chicken pieces onto skewers and place them on a broi

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare the tofu or paneer to use in this recipe:\n\nFor Tofu:\n- Use extra-firm tofu and press it to remove excess moisture. To do this, wrap the block of tofu in a clean kitchen towel or paper towels, place it on a plate, and then place a heavy object on top (like a cast iron skillet or a few heavy books). Let it sit for about 30 minutes, then cut it into bite-sized pieces and proceed with the recipe as you would with the chicken.\n\nFor Paneer:\n- If you prefer to use paneer, you can either make it at home or purchase it from a store. Cut the paneer into bite-sized pieces and proceed with the recipe as you would with the chicken.\n\nBy using tofu or paneer as a substitute for the chicken, you can enjoy a delicious vegetarian version of Chicken Tikka Masala.")